In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [30]:
from numpy import genfromtxt

def load_list(filename):
    vocabulary = []
    with open(filename, 'r') as f:
        for l in f:
            vocabulary.append(l.strip())
    return np.asarray(vocabulary)

pos_related = load_list('./sentence_data/pos_related.txt')
neg_related = load_list('./sentence_data/neg_related.txt')
pos_unrelated = load_list('./sentence_data/pos_unrelated.txt')
neg_unrelated = load_list('./sentence_data/neg_unrelated.txt')

In [31]:
print(pos_related.shape)
print(neg_related.shape)
print(pos_unrelated.shape)
print(neg_unrelated.shape)

(466,)
(83,)
(388,)
(34,)


In [25]:
print(pos_related[:10])

['not perfect by a long shot, but definitely good for a smile on a bad day.'
 'the whole cast was great, each character had their own personality and charm.'
 'even though it has one of the standard "revenge price plot," this film is my favorite of vincent price\'s work.'
 'i really enjoyed this movie, it is really fun to watch get elvira into all these adventure, she is just great.'
 'with more laugh than any other third-in-a-disney-series movie, hakuna matata is worth watching - if only for the hot tub scene which is still funny despite being a little bit predictable.'
 'it is really a wonderful thriller i enjoyed very much'
 'when my sister said this movie was gonna be good i had second thought but i watched it and it was actually funny'
 'it touched me in a way that, even all these year later, still affects me.'
 'i strongly recommend seeing for all'
 'without a doubt, the best late night television ever.']


In [29]:
print(neg_related[:10])

['i had numerous problem with this film'
 'dear god i do not know where to start why this movie sucked too much'
 'i was pretty disappointed'
 'if you are tempted to watch this movie, rip your eyeball out and flush them down the toilet'
 'the music there was was annoying, and boring'
 'someone must have been seriously joking when they made this film'
 'ugly then, uglier now' 'this film is predictable'
 'even the supporting male character are all "bad"'
 'trust me, this is one let down movie that you want to avoid and this comes from one huge denzel washington fan']


In [44]:
related_set = np.hstack((pos_related, neg_related))
print(related_set.shape)
y_related = np.ones(related_set.shape)
print(y_related.shape)

(549,)
(549,)


In [46]:
unrelated_set = np.hstack((pos_unrelated, neg_unrelated))
print(unrelated_set.shape)
y_unrelated = np.zeros(unrelated_set.shape)
print(y_unrelated.shape)

(422,)
(422,)


In [49]:
X = np.hstack((related_set, unrelated_set))
y = np.hstack((y_related, y_unrelated))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [53]:
print(y_train.shape)
print(y_test.shape)

(650,)
(321,)


In [57]:
from sklearn.feature_extraction.text import CountVectorizer

token = r"(?u)\b[\w\'/]+\b"
tf_vectorizer = CountVectorizer(lowercase=True, max_df=1.0, min_df=1, binary=False, token_pattern=token)
tf_vectorizer.set_params(ngram_range=(1,1))

X_vectorized = tf_vectorizer.fit_transform(X)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern="(?u)\\b[\\w\\'/]+\\b",
        tokenizer=None, vocabulary=None)

In [56]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)

print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

ValueError: could not convert string to float: 'with her fear of audience and dislike of such performing, this may have been the toughest part for her, but if so, to her credit it does not show'